##Preliminary Notes

The aim of the InCrediblAE shared task is to build your own custom attack method that will generate adversarial examples to fool a victim classifier. This notebook is intended as an easy way for you to get started.

<br>

### Using GPU
It is recommended that you run this notebook with a GPU. To do this, click on "additional connection options" (next to Connect / RAM usage), select "change runtime type", and select a GPU.

<br>

### (optional) Mounting Google Drive - don't bother with this if running this notebook for first time
If you will be re-running this notebook many times, it might be convenient to mount your personal google drive. This will allow you to
1. load data/victim files quickly rather than re-downloading them with each session
2. save output files to a permanent location

Instructions for mounting are in the 'Making your own attack section'.


# Setup (installing dependencies)

In [1]:
!git clone https://github.com/piotrmp/BODEGA

Cloning into 'BODEGA'...
remote: Enumerating objects: 149, done.
remote: Counting objects: 100% (146/146), done.
remote: Compressing objects: 100% (97/97), done.
remote: Total 149 (delta 79), reused 112 (delta 49), pack-reused 3
Receiving objects: 100% (149/149), 33.93 KiB | 2.61 MiB/s, done.
Resolving deltas: 100% (79/79), done.


In [2]:
%pip install OpenAttack
%pip install editdistance
%pip install bert-score
%pip install git+https://github.com/lucadiliello/bleurt-pytorch.git


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 145.4/145.4 kB 2.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 6.5 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manyl

In [3]:
!git clone https://gitlab.clarin-pl.eu/syntactic-tools/lambo.git
%pip install ./lambo


Cloning into 'lambo'...
remote: Enumerating objects: 441, done.
remote: Counting objects: 100% (129/129), done.
remote: Compressing objects: 100% (120/120), done.
remote: Total 441 (delta 67), reused 0 (delta 0), pack-reused 312
Receiving objects: 100% (441/441), 135.99 KiB | 400.00 KiB/s, done.
Resolving deltas: 100% (241/241), done.
Processing ./lambo
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.8/143.8 kB 3.6 MB/s eta 0:00:00
  Created wheel for lambo: filename=lambo-2.3-py3-none-any.whl size=92978 sha256=ea6e237af66d34112c9bd7fcabea3fe2a64d253ccd86b03aea1f33e959a8fa94
  Stored in directory: /tmp/pip-ephem-wheel-cache-j0nuidmf/wheels/e6/9c/4d/38b46f623fbde94d97d9a8c0f9c10f2265161c844c2940c95f
Successfully built lambo
  Attempting uninstall: urllib3
    Found existing installation: urllib3 2.0.7
    Uninstalling urllib3-2.0.7:
      Successfully 

# Downloading victim models and data

Data and models are downloaded by cloning the [clef2024-checkthat repo](https://gitlab.com/checkthat_lab/clef2024-checkthat-lab.git)
* alternative [google drive folder link](https://drive.google.com/drive/folders/1ZsDHSejiv4USae0viTsfeLpvqXdeq0FL?usp=sharing)

Data and models are downloaded then moved to /content/BODEGA/incrediblAE_public_release

In [4]:
# temporary folder for downloading victim models and data
! mkdir /content/clef2024-checkthat-lab

import os, sys
os.chdir("/content/clef2024-checkthat-lab")

! git init
! git remote add -f origin https://gitlab.com/checkthat_lab/clef2024-checkthat-lab.git
! git sparse-checkout init
! git sparse-checkout set "task6/incrediblAE_public_release"
! git pull origin main


hint: Using 'master' as the name for the initial branch. This default branch name
hint: is subject to change. To configure the initial branch name to use in all
hint: of your new repositories, which will suppress this warning, call:
hint: 
hint: 	git config --global init.defaultBranch <name>
hint: 
hint: Names commonly chosen instead of 'master' are 'main', 'trunk' and
hint: 'development'. The just-created branch can be renamed via this command:
hint: 
hint: 	git branch -m <name>
Initialized empty Git repository in /content/clef2024-checkthat-lab/.git/
Updating origin
remote: Enumerating objects: 840, done.
remote: Counting objects: 100% (757/757), done.
remote: Compressing objects: 100% (476/476), done.
remote: Total 840 (delta 443), reused 445 (delta 277), pack-reused 83 (from 1)
Receiving objects: 100% (840/840), 12.59 MiB | 21.56 MiB/s, done.
Resolving deltas: 100% (451/451), done.
From https://gitlab.com/checkthat_lab/clef2024-checkthat-lab
 * [new branch]      FHaouari-main-patch

In [5]:
# move downloaded files to /content/BODEGA
! mv /content/clef2024-checkthat-lab/task6/incrediblAE_public_release /content/BODEGA/incrediblAE_public_release

Misc set up

In [6]:
#folder for storing results of attack method
! mkdir /content/BODEGA/outputs

#code below assumes we are working from the BODEGA repo
os.chdir("/content/BODEGA")

Below is an alternative way to download the surprise models (in case the gitlab repo doesn't work)

In [ ]:
# !gdown https://drive.google.com/drive/folders/1PT9f_WH3D5N1kKPXZgta0-C4oHn2rFeX?usp=sharing -O /tmp/folder --folder


In [ ]:
# !mv /tmp/folder/FC_surprise-512.pth /content/BODEGA/incrediblAE_public_release/FC/surprise-512.pth
# !mv /tmp/folder/HN_surprise-512.pth /content/BODEGA/incrediblAE_public_release/HN/surprise-512.pth
# !mv /tmp/folder/PR2_surprise-512.pth /content/BODEGA/incrediblAE_public_release/PR2/surprise-512.pth
# !mv /tmp/folder/RD_surprise-512.pth /content/BODEGA/incrediblAE_public_release/RD/surprise-512.pth
# !mv /tmp/folder/C19_surprise-512.pth /content/BODEGA/incrediblAE_public_release/C19/surprise-512.pth



# Making your own attack

## Imports

In [7]:
import gc
import os
import pathlib
import sys
import time
import random
import numpy as np

import OpenAttack
import torch
import datasets
from datasets import Dataset

from OpenAttack.tags import Tag
from OpenAttack.text_process.tokenizer import PunctTokenizer

from metrics.BODEGAScore import BODEGAScore
from utils.data_mappings import dataset_mapping, dataset_mapping_pairs, SEPARATOR_CHAR
from utils.no_ssl_verify import no_ssl_verify
from victims.bert import VictimBERT
from victims.bert import readfromfile_generator as BERT_readfromfile_generator
from victims.bilstm import VictimBiLSTM
from victims.caching import VictimCache
from victims.unk_fix_wrapper import UNK_TEXT

#imports for BodegaAttackEval wrapper
from typing import Any, Dict, Generator, Iterable, List, Optional, Union
from tqdm import tqdm
from OpenAttack.utils import visualizer, result_visualizer, get_language, language_by_name
from OpenAttack.tags import *

Definining suprise classifier class

In [8]:
import numpy

from datasets import Dataset, DatasetDict, concatenate_datasets
from transformers import AutoTokenizer, DataCollatorWithPadding, AutoConfig
from transformers import AutoModelForSequenceClassification
from torch.utils.data import DataLoader
from tqdm.auto import tqdm

from utils.data_mappings import SEPARATOR
import pathlib

BATCH_SIZE = 16
MAX_LEN = 512
EPOCHS = 5
MAX_BATCHES = -1
pretrained_model = "roberta-base"

def trim(text, tokenizer):
    offsets = tokenizer(text, truncation=True, max_length=MAX_LEN + 10, return_offsets_mapping=True)['offset_mapping']
    limit = len(text)
    if len(offsets) > MAX_LEN:
        limit = offsets[512][1]
    return text[:limit]


def roberta_readfromfile_generator(subset, dir, with_pairs=False, trim_text=False):
    tokenizer = AutoTokenizer.from_pretrained(pretrained_model)
    for line in open(dir / (subset + '.tsv')):
        parts = line.split('\t')
        label = int(parts[0])
        if not with_pairs:
            text = parts[2].strip().replace('\\n', '\n').replace('\\t', '\t').replace('\\\\', '\\')
            if trim_text:
                text = trim(text, tokenizer)
            yield {'fake': label, 'text': text}
        else:
            text1 = parts[2].strip().replace('\\n', '\n').replace('\\t', '\t').replace('\\\\', '\\')
            text2 = parts[3].strip().replace('\\n', '\n').replace('\\t', '\t').replace('\\\\', '\\')
            if trim_text:
                text1 = trim(text1, tokenizer)
                text2 = trim(text2, tokenizer)
            yield {'fake': label, 'text1': text1, 'text2': text2}


def eval_loop(model, eval_dataloader, device, skip_visual=False):
    print("Evaluating...")
    model.eval()
    progress_bar = tqdm(range(len(eval_dataloader)), ascii=True, disable=skip_visual)
    correct = 0
    size = 0
    TPs = 0
    FPs = 0
    FNs = 0
    for i, batch in enumerate(eval_dataloader):
        batch = {k: v.to(device) for k, v in batch.items()}
        with torch.no_grad():
            outputs = model(**batch)

        logits = outputs.logits
        # print(logits)
        # a = input()
        pred = torch.argmax(logits, dim=-1).detach().to(torch.device('cpu')).numpy()
        Y = batch["labels"].to(torch.device('cpu')).numpy()
        eq = numpy.equal(Y, pred)
        size += len(eq)
        correct += sum(eq)
        TPs += sum(numpy.logical_and(numpy.equal(Y, 1.0), numpy.equal(pred, 1.0)))
        FPs += sum(numpy.logical_and(numpy.equal(Y, 0.0), numpy.equal(pred, 1.0)))
        FNs += sum(numpy.logical_and(numpy.equal(Y, 1.0), numpy.equal(pred, 0.0)))
        progress_bar.update(1)

        # print(Y)
        # print(pred)
        # a = input()

        if i == MAX_BATCHES:
            break
    print('Accuracy: ' + str(correct / size))
    print('F1: ' + str(2 * TPs / (2 * TPs + FPs + FNs)))
    print(correct, size, TPs, FPs, FNs)

    results = {
        'Accuracy': correct/size,
        'F1': 2 * TPs / (2 * TPs + FPs + FNs)
    }
    return results


class VictimRoBERTa(OpenAttack.Classifier):
    def __init__(self, path, task, device=torch.device('cpu')):
        self.device = device
        config = AutoConfig.from_pretrained(pretrained_model)
        self.model = AutoModelForSequenceClassification.from_config(config)
        self.model.load_state_dict(torch.load(path))
        self.model.to(device)
        self.model.eval()
        self.tokenizer = AutoTokenizer.from_pretrained(pretrained_model)
        self.with_pairs = (task == 'FC' or task == 'C19')

    def get_pred(self, input_):
        return self.get_prob(input_).argmax(axis=1)

    def get_prob(self, input_):
        try:
            probs = None
            # print(len(input_), input_)

            batched = [input_[i * BATCH_SIZE:(i + 1) * BATCH_SIZE] for i in
                       range((len(input_) + BATCH_SIZE - 1) // BATCH_SIZE)]
            for batched_input in batched:
                if not self.with_pairs:
                    tokenised = self.tokenizer(batched_input, truncation=True, padding=True, max_length=MAX_LEN,
                                               return_tensors="pt")
                else:
                    parts = [x.split(SEPARATOR) for x in batched_input]
                    tokenised = self.tokenizer([x[0] for x in parts], [(x[1] if len(x) == 2 else '') for x in parts],
                                               truncation=True, padding=True,
                                               max_length=MAX_LEN,
                                               return_tensors="pt")
                with torch.no_grad():
                    tokenised = {k: v.to(self.device) for k, v in tokenised.items()}
                    outputs = self.model(**tokenised)
                probs_here = torch.nn.functional.softmax(outputs.logits, dim=-1).to(torch.device('cpu')).numpy()
                if probs is not None:
                    probs = numpy.concatenate((probs, probs_here))
                else:
                    probs = probs_here
            return probs
        except Exception as e:
            # Used for debugging
            raise


In [9]:
using_mounted_drive = False
print('Cuda device available', torch.cuda.is_available())

Cuda device available True


## (do not change) Wrapper for producing submission file

In [10]:
class BodegaAttackEval(OpenAttack.AttackEval):
  '''
  wrapper for OpenAttack.AttackEval to produce a submission.tsv file for shared task evaluation

  To perform evaluation, we use a new method: eval_and_save_tsv() rather than the usual AttackEval.eval()
  submission.tsv file consists of 4 columns for each sample in attack set: succeeded, num_queries, original_text and modified text (newlines are escaped)

  '''
  def eval_and_save_tsv(self, dataset: Iterable[Dict[str, Any]], total_len : Optional[int] = None, visualize : bool = False, progress_bar : bool = False, num_workers : int = 0, chunk_size : Optional[int] = None, tsv_file_path: Optional[os.PathLike] = None):
      """
      Evaluation function of `AttackEval`.

      Args:
          dataset: An iterable dataset.
          total_len: Total length of dataset (will be used if dataset doesn't has a `__len__` attribute).
          visualize: Display a pretty result for each data in the dataset.
          progress_bar: Display a progress bar if `True`.
          num_workers: The number of processes running the attack algorithm. Default: 0 (running on the main process).
          chunk_size: Processing pool trunks size.

          tsv_file_path: path to save submission tsv

      Returns:
          A dict of attack evaluation summaries.

      """


      if hasattr(dataset, "__len__"):
          total_len = len(dataset)

      def tqdm_writer(x):
          return tqdm.write(x, end="")

      if progress_bar:
          result_iterator = tqdm(self.ieval(dataset, num_workers, chunk_size), total=total_len)
      else:
          result_iterator = self.ieval(dataset, num_workers, chunk_size)

      total_result = {}
      total_result_cnt = {}
      total_inst = 0
      success_inst = 0

      #list for tsv
      x_orig_list = []
      x_adv_list = []
      num_queries_list = []
      succeed_list = []

      # Begin for
      for i, res in enumerate(result_iterator):
          total_inst += 1
          success_inst += int(res["success"])

          if TAG_Classification in self.victim.TAGS:
              x_orig = res["data"]["x"]
              if res["success"]:
                  x_adv = res["result"]
                  if Tag("get_prob", "victim") in self.victim.TAGS:
                      self.victim.set_context(res["data"], None)
                      try:
                          probs = self.victim.get_prob([x_orig, x_adv])
                      finally:
                          self.victim.clear_context()
                      y_orig = probs[0]
                      y_adv = probs[1]
                  elif Tag("get_pred", "victim") in self.victim.TAGS:
                      self.victim.set_context(res["data"], None)
                      try:
                          preds = self.victim.get_pred([x_orig, x_adv])
                      finally:
                          self.victim.clear_context()
                      y_orig = int(preds[0])
                      y_adv = int(preds[1])
                  else:
                      raise RuntimeError("Invalid victim model")
              else:
                  y_adv = None
                  x_adv = None
                  if Tag("get_prob", "victim") in self.victim.TAGS:
                      self.victim.set_context(res["data"], None)
                      try:
                          probs = self.victim.get_prob([x_orig])
                      finally:
                          self.victim.clear_context()
                      y_orig = probs[0]
                  elif Tag("get_pred", "victim") in self.victim.TAGS:
                      self.victim.set_context(res["data"], None)
                      try:
                          preds = self.victim.get_pred([x_orig])
                      finally:
                          self.victim.clear_context()
                      y_orig = int(preds[0])
                  else:
                      raise RuntimeError("Invalid victim model")
              info = res["metrics"]
              info["Succeed"] = res["success"]
              if visualize:
                  if progress_bar:
                      visualizer(i + 1, x_orig, y_orig, x_adv, y_adv, info, tqdm_writer, self.tokenizer)
                  else:
                      visualizer(i + 1, x_orig, y_orig, x_adv, y_adv, info, sys.stdout.write, self.tokenizer)

              #list for tsv
              succeed_list.append(res["success"])
              num_queries_list.append(res["metrics"]["Victim Model Queries"])
              x_orig_list.append(x_orig)

              if res["success"]:
                x_adv_list.append(x_adv)
              else:
                x_adv_list.append("ATTACK_UNSUCCESSFUL")



          for kw, val in res["metrics"].items():
              if val is None:
                  continue

              if kw not in total_result_cnt:
                  total_result_cnt[kw] = 0
                  total_result[kw] = 0
              total_result_cnt[kw] += 1
              total_result[kw] += float(val)
      # End for

      summary = {}
      summary["Total Attacked Instances"] = total_inst
      summary["Successful Instances"] = success_inst
      summary["Attack Success Rate"] = success_inst / total_inst
      for kw in total_result_cnt.keys():
          if kw in ["Succeed"]:
              continue
          if kw in ["Query Exceeded"]:
              summary["Total " + kw] = total_result[kw]
          else:
              summary["Avg. " + kw] = total_result[kw] / total_result_cnt[kw]

      if visualize:
          result_visualizer(summary, sys.stdout.write)


      #saving tsv
      if tsv_file_path is not None:
        with open(tsv_file_path, 'w') as f:
          f.write('succeeded' + '\t' + 'num_queries' + '\t' + 'original_text' + '\t' + 'modified_text' + '\t'+ '\n') #header
          for success, num_queries, x_orig, x_adv in zip(succeed_list, num_queries_list, x_orig_list, x_adv_list):
            escaped_x_orig = x_orig.replace('\n', '\\n') #escaping newlines
            escaped_x_adv = x_adv.replace('\n', '\\n')
            f.write(str(success) + '\t' + str(num_queries) + '\t' + escaped_x_orig + '\t' + escaped_x_adv + '\t'+ '\n')

      return summary

## (optional) Mounting Google Drive


Steps to use mounted google drive:
1. create a folder in your local google drive (e.g. `incrediblAE_public_release`)  
2. download all directories from the download link (see [Download section above](https://colab.research.google.com/drive/1juHWIL44z8O3C5wDAE45vzlJgX51KI5D?authuser=3#scrollTo=eVVE2-64rKuS&line=3&uniqifier=1://)) and upload them to your google drive folder
3. create an empty subdirectory called `outputs` (`incredibleAE_public_release/outputs/`)

At this point, your google drive folder should have 6 subdirectories (C19, FC, HN, PR2, RD, and outputs)
4. uncomment code below, replacing path_to_mounted_dir with path to your folder (e.g. `/content/drive/My Drive/incrediblAE_public_release`)



In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

# using_mounted_drive = True
# path_to_mounted_folder = '/content/drive/My Drive/incrediblAE_public_release'


In [ ]:

# references = ["This is a test."]
# candidates = ["This is the test."]


# scores = scorer.score(references=references, candidates=candidates)
# assert isinstance(scores, list) and len(scores) == 1
# print(scores)

In [ ]:
# !git clone https://github.com/google-research/bleurt.git
# %cd bleurt
# !pip install .

In [ ]:
# from bleurt import score

# checkpoint = "bleurt/test_checkpoint"
# scorer = score.BleurtScorer(checkpoint)

You can also comment out the !gdown command in Downloading section, so the notebook doesn't redownload data each time you run it.

## Making custom attacker (token shuffler)

Here's an example of how to create a custom attack method.
Your attacker will need to subclass `OpenAttack.attackers.ClassificationAttacker`  

(See also OpenAttack framework docs: https://openattack.readthedocs.io/en/latest/)

In [11]:
'''
Contextual Embeddings
'''
import copy
from transformers import BertConfig, BertTokenizerFast, BertForMaskedLM
from transformers import RobertaConfig, RobertaTokenizerFast, RobertaForMaskedLM

import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')

stop_words = set(stopwords.words('english'))

# from bleurt import score



# from transformers import RobertaConfig, RobertaModel

class MyAttacker(OpenAttack.attackers.ClassificationAttacker):
    @property
    def TAGS(self):
        return { self.__lang_tag, Tag("get_pred", "victim"), Tag("get_prob", "victim") }

    def __init__(self,
            # mlm_path : str = 'bert-base-uncased',
            mlm_path : str = 'roberta-base',
            k : int = 36,
            use_bpe : int = 0,
            threshold_pred_score : float = 0.3,
            max_length : int = 512,
            device : Optional[torch.device] = None,
            filter_words : List[str] = stop_words
        ):
        # self.tokenizer_mlm = BertTokenizerFast.from_pretrained(mlm_path, do_lower_case=True)
        self.tokenizer_mlm = RobertaTokenizerFast.from_pretrained(mlm_path, do_lower_case=True)
        if device is not None:
            self.device = device
        else:
            self.device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
        # config_atk = BertConfig.from_pretrained(mlm_path)
        # self.mlm_model = BertForMaskedLM.from_pretrained(mlm_path, config=config_atk).to(self.device)

        config_atk = RobertaConfig.from_pretrained(mlm_path)
        self.mlm_model = RobertaForMaskedLM.from_pretrained(mlm_path, config=config_atk).to(self.device)

        self.k = k
        self.use_bpe = use_bpe
        self.threshold_pred_score = threshold_pred_score
        self.max_length = max_length
        # self.tokenizer = PunctTokenizer()
        # self.tokenizer_mlm = BertTokenizerFast.from_pretrained(mlm_path, do_lower_case=True)

        self.filter_words = filter_words

        self.__lang_tag = TAG_English
        # if filter_words is None:
        #     filter_words = get_default_filter_words(self.__lang_tag)
        # self.filter_words = set(filter_words)



    def attack(self, victim, input_, goal):
        x_orig = input_.lower()

        # return None
        tokenizer = self.tokenizer_mlm
        # MLM-process
        words, sub_words, keys = self._tokenize(x_orig, tokenizer)
        max_length = self.max_length
        # original label
        inputs = tokenizer.encode_plus(x_orig, None, add_special_tokens=True, max_length=max_length, truncation=True)
        # input_ids, _ = torch.tensor(inputs["input_ids"]), torch.tensor(inputs["token_type_ids"])
        input_ids = torch.tensor(inputs["input_ids"])

        orig_probs = torch.Tensor(victim.get_prob([x_orig]))
        orig_probs = orig_probs[0].squeeze()

        current_prob = orig_probs.max()

        sub_words = ['[CLS]'] + sub_words[:2] + sub_words[2:max_length - 2] + ['[SEP]']

        input_ids_ = torch.tensor([tokenizer.convert_tokens_to_ids(sub_words)])



        word_predictions_orig = self.mlm_model(input_ids_.to(self.device))[0].squeeze()  # seq-len(sub) vocab
        word_pred_scores_all, word_predictions = torch.topk(word_predictions_orig, self.k, -1)  # seq-len k

        word_predictions = word_predictions[1:len(sub_words) + 1, :]
        word_pred_scores_all = word_pred_scores_all[1:len(sub_words) + 1, :]

        # Generate a score indicating the importance of each words to the classification output?
        # Change a word, if probability distribution changes a lot, than the important score will be higher
        important_scores = self.get_important_scores(words, victim, current_prob, goal.target, orig_probs)

        # Set the words with higher important score at the begining.
        list_of_index = sorted(enumerate(important_scores), key=lambda x: x[1], reverse=True)
        final_words = copy.deepcopy(words)

        # final_adverse_check = tokenizer.convert_tokens_to_string(final_words)

        final_adverse_check = " ".join(final_words)

        contextual_embeddings = []
        try:
          # contextual_embeddings
          contextual_embeddings = self.get_contextual_embeddings(input_ids_)[0]
        except:
          print("Too long")

        cnt = 0
        for top_index in list_of_index:
          # print(cnt)
          if cnt > 50:
            break
          cnt += 1
          tgt_word = words[top_index[0]]

          if keys[top_index[0]][0] > max_length - 2:
                continue
          # position = top_index[0]
          # position = keys[top_index[0]][0]:keys[top_index[0]][1]
          position = [i for i in range(keys[top_index[0]][0],keys[top_index[0]][1]+1)]

          substitutes = word_predictions[keys[top_index[0]][0]:keys[top_index[0]][1]]  # L, k 获取当前目标词的真正位置，因为用BPE tokenized，序列长度变大了


          word_pred_scores = word_pred_scores_all[keys[top_index[0]][0]:keys[top_index[0]][1]]

          # use subword level substitution

          substitutes = self.get_substitues(substitutes, tokenizer, self.mlm_model, self.use_bpe, word_pred_scores, self.threshold_pred_score)
            # substitutes = torch.tensor([[tokenizer.convert_tokens_to_ids(sub) for sub in substitutes ]]).to(self.device)

          effective_substitutes = []
          if len(contextual_embeddings) != 0:
            try:
              # Assume 'position' is the index of the word in the input_ids that you want to attack
              target_embedding = contextual_embeddings[keys[top_index[0]][0]:keys[top_index[0]][1]]
              aggregated_embedding = torch.mean(target_embedding, dim=0, keepdim=True)

              # Generate potential substitutes from MLM

              top_k_words = torch.topk(word_predictions_orig[keys[top_index[0]][0]:keys[top_index[0]][1]], self.k,-1).indices

              # top_k_words = substitutes[0]
              substitutes_tokens = [tokenizer._convert_id_to_token(idx.item()) for idx in top_k_words[-1]]


              # The original substitutes are candidates after the target word predicted by LM, while MyAttacker is based on contextual embeddings

              # Get embeddings for substitutes
              substitute_embeddings = self.get_embeddings_for_substitutes(x_orig,position,substitutes_tokens)


              # Calculate cosine similarities and filter based on threshold
              similarities = self.contextual_cosine_similarity(aggregated_embedding,substitute_embeddings)
              effective_substitutes = [substitutes_tokens[idx] for idx, sim in enumerate(similarities) if sim > self.threshold_pred_score]
            except:
              print("Too long")
          # orig_substitutes = [tokenizer._convert_id_to_token(sub.item())for sub in substitutes[0]]

          for i in range(len(effective_substitutes)):
            if 'Ġ' not in effective_substitutes[i]:
              effective_substitutes[i] = 'Ġ' + effective_substitutes[i]

          for i in range(len(substitutes)):
            if 'Ġ' not in substitutes[i]:
              substitutes[i] = 'Ġ' + substitutes[i]

          # print(effective_substitutes)
          effective_substitutes_new = [self.tokenizer_mlm.convert_tokens_to_string([e]) for e in effective_substitutes]
          substitutes_new = [self.tokenizer_mlm.convert_tokens_to_string([e]) for e in substitutes]
          # print(effective_substitutes_new)
          # print(effective_substitutes_new)

          # Union two candidate lists
          effective_substitutes_union_set = set(substitutes_new).union(set(effective_substitutes_new))
          effective_substitutes_union = list(effective_substitutes_union_set)

          final_adverse = x_orig

          most_gap = 0.0
          candidate = None
          # # If there is not candidates from contextual embeddings than use the original BERTAttacker
          # if len(effective_substitutes) == 0:
          #   print("*"*10)
            # effective_substitutes = orig_substitutes[1:]

          # # sort the substitue by semantic score
          # tgt = final_adverse
          # resort = []

          # scorer = score.BleurtScorer(checkpoint)
          # # scores = scorer.score(references=references, candidates=candidates)


          # for substitute in effective_substitutes_union:
          #   original_tokens = self.tokenizer_mlm.tokenize(tgt)
          #   substitute_token = self.tokenizer_mlm.tokenize(substitute)
          #   modified_tokens = original_tokens[:position[0]] + substitute_token + original_tokens[position[-1]:]

          #   modified_new_tokens = self.tokenizer_mlm.convert_tokens_to_string(modified_tokens)

          #   bleurt_score = scorer.score(references=[tgt], candidates=[modified_new_tokens])[0]
          #   # bleurt_score = bleurt.compute(predictions=" ".join(modified_tokens), references=tgt)['scores']
          #   resort.append([bleurt_score,substitute])

          # resort.sort(key=lambda x:x[0],reverse=True)
          # print(resort)
          # resort_substitutes = [x[1] for x in resort]
          # print(resort_substitutes)

          for substitute in effective_substitutes_union:
                if substitute == tgt_word:
                    continue  # filter out original word
                if '##' in substitute:
                    continue  # filter out sub-word

                if substitute in self.filter_words:
                    continue


                temp_replace = final_words
                temp_replace[top_index[0]] = substitute
                # temp_text = tokenizer.convert_tokens_to_string(temp_replace)
                temp_text = " ".join(temp_replace)
                inputs = tokenizer.encode_plus(temp_text, None, add_special_tokens=True, max_length=max_length, truncation=True)
                input_ids = torch.tensor(inputs["input_ids"]).unsqueeze(0).to(self.device)
                seq_len = input_ids.size(1)

                temp_prob = torch.Tensor(victim.get_prob([temp_text]))[0].squeeze()
                temp_label = torch.argmax(temp_prob)

                # if goal.check(final_adverse, temp_label):
                if goal.check(temp_text,temp_label):
                    final_words[top_index[0]] = substitute
                    # feature.changes.append([keys[top_index[0]][0], substitute, tgt_word])
                    final_adverse = temp_text
                    # feature.success = 4
                    return final_adverse
                else:
                    label_prob = temp_prob[goal.target]
                    gap = current_prob - label_prob
                    if gap > most_gap:
                        most_gap = gap
                        candidate = substitute

          if most_gap > 0:
              # feature.change += 1
              # feature.changes.append([keys[top_index[0]][0], candidate, tgt_word])
              current_prob = current_prob - most_gap
              final_words[top_index[0]] = candidate

        final_adverse = tokenizer.convert_tokens_to_string(final_words)


        return None

    def get_embeddings_for_substitutes(self, sentence, position, substitutes_tokens):
        # This function will replace the target word at 'position' in 'sentence' with each substitute and generate embeddings
        embeddings = []
        original_tokens = self.tokenizer_mlm.tokenize(sentence)

        for substitute in substitutes_tokens:
            modified_tokens = original_tokens[:position[0]] + [substitute] + original_tokens[position[-1]:]
            # input_ids =self.tokenizer_mlm.encode_plus(modified_tokens, None, add_special_tokens=True, max_length=self.max_length, truncation=True)
            # input_tensor = torch.tensor(input_ids["input_ids"]).to(self.device)
            input_ids = self.tokenizer_mlm.convert_tokens_to_ids(modified_tokens[:self.max_length-2])
            input_ids = self.tokenizer_mlm.build_inputs_with_special_tokens(input_ids)
            input_tensor = torch.tensor([input_ids]).to(self.device)

            with torch.no_grad():
                outputs = self.mlm_model(input_tensor, output_hidden_states=True)
                hidden_states = outputs.hidden_states

                # We assume we're interested in the embedding of the word at 'position'
                # Still have some samples out of index
                # embedding = hidden_states[-2][0, position[-1]]  # Adjusting index for special tokens
                # embeddings.append(embedding)

                embedding = hidden_states[-2][0][position[0]:position[-1]]
                aggregated_embedding = torch.mean(embedding, dim=0, keepdim=True)
                embeddings.append(aggregated_embedding)


        return embeddings

    def get_contextual_embeddings(self, input_ids):
        """ Get the contextual embeddings from BERT model for input IDs. """
        with torch.no_grad():
            outputs = self.mlm_model(input_ids=input_ids.to(self.device), output_hidden_states=True)
            hidden_states = outputs.hidden_states

        return hidden_states[-2]  # using second-to-last layer

    def contextual_cosine_similarity(self, target_embedding, embeddings):
        # Compute cosine similarities between the target embedding and each embedding in embeddings
        similarities = [torch.cosine_similarity(target_embedding, embedding, dim=1).item() for embedding in embeddings]
        return similarities

    def _tokenize(self, seq, tokenizer):
        seq = seq.replace('\n', '').lower()
        words = seq.split(' ')

        sub_words = []
        keys = []
        index = 0
        for word in words:
            sub = tokenizer.tokenize(word)
            sub_words += sub
            keys.append([index, index + len(sub)])
            index += len(sub)

        return words, sub_words, keys

    def _get_masked(self, words):
          len_text = max(len(words), 2)
          masked_words = []
          for i in range(len_text - 1):
              masked_words.append(words[0:i] + ['[UNK]'] + words[i + 1:])
          # list of words
          return masked_words

    def get_important_scores(self, words, tgt_model, orig_prob, orig_label, orig_probs):
          masked_words = self._get_masked(words)
          texts = [' '.join(words) for words in masked_words]  # list of text of masked words
          leave_1_probs = torch.Tensor(tgt_model.get_prob(texts))
          leave_1_probs_argmax = torch.argmax(leave_1_probs, dim=-1)

          import_scores = (orig_prob
                          - leave_1_probs[:, orig_label]
                          +
                          (leave_1_probs_argmax != orig_label).float()
                          * (leave_1_probs.max(dim=-1)[0] - torch.index_select(orig_probs, 0, leave_1_probs_argmax))
                          ).data.cpu().numpy()

          return import_scores

    def get_substitues(self, substitutes, tokenizer, mlm_model, use_bpe, substitutes_score=None, threshold=3.0):
          # substitues L,k
          # from this matrix to recover a word
          words = []
          sub_len, k = substitutes.size()  # sub-len, k

          if sub_len == 0:
              return words

          elif sub_len == 1:
              for (i,j) in zip(substitutes[0], substitutes_score[0]):
                  if threshold != 0 and j < threshold:
                      break
                  words.append(tokenizer._convert_id_to_token(int(i)))
          else:
              if use_bpe == 1:
                  words = self.get_bpe_substitues(substitutes, tokenizer, mlm_model)
              else:
                  return words
          return words
    def get_bpe_substitues(self, substitutes, tokenizer, mlm_model):
          # substitutes L, k

          substitutes = substitutes[0:12, 0:4] # maximum BPE candidates

          # find all possible candidates
          all_substitutes = []
          for i in range(substitutes.size(0)):
              if len(all_substitutes) == 0:
                  lev_i = substitutes[i]
                  all_substitutes = [[int(c)] for c in lev_i]
              else:
                  lev_i = []
                  for all_sub in all_substitutes:
                      for j in substitutes[i]:
                          lev_i.append(all_sub + [int(j)])
                  all_substitutes = lev_i

          # all substitutes  list of list of token-id (all candidates)
          c_loss = torch.nn.CrossEntropyLoss(reduction='none')
          word_list = []
          # all_substitutes = all_substitutes[:24]
          all_substitutes = torch.tensor(all_substitutes) # [ N, L ]
          all_substitutes = all_substitutes[:24].to(self.device)
          # print(substitutes.size(), all_substitutes.size())
          N, L = all_substitutes.size()
          word_predictions = mlm_model(all_substitutes)[0] # N L vocab-size

          ppl = c_loss(word_predictions.view(N*L, -1), all_substitutes.view(-1)) # [ N*L ]
          ppl = torch.exp(torch.mean(ppl.view(N, L), dim=-1)) # N
          _, word_list = torch.sort(ppl)
          word_list = [all_substitutes[i] for i in word_list]
          final_words = []
          for word in word_list:
              tokens = [tokenizer._convert_id_to_token(int(i)) for i in word]
              text = tokenizer.convert_tokens_to_string(tokens)
              final_words.append(text)

          return final_words




[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


## Testing your attack

The code below will test MyAttacker (above) on the victim classifier, compute BODEGA score, and output results to /content/BODEGA/outputs.

WARNING: files in default output directory (/content/BODGEa/outputs) do not persist after you disconnect from the colab runtime session. To keep them, you can either:

1. download them manually or
2. set `out_dir` to a mounted Google Drive directory (will automatically save files to your google drive)



### Choose task + victim classifier

In [19]:
# determinism
random.seed(10)
torch.manual_seed(10)
np.random.seed(0)

# Change these variables to what you want
task = 'RD' # PR2, HN, FC, RD, C19
victim_model = 'BERT' # BERT or BiLSTM or surprise
using_custom_attacker = True # change to False if you want to test out OpenAttack's pre-implemented attackers (e.g. BERTattack)
attack = 'RoBERTa' # if using custom attack, this name can be whatever you want. If using pre-implemented attack, set to name of attacker ('BERTattack')

# misc variables - no need to change
targeted = False # this shared task evaluates performance in an untargeted scenario
visualize_adv_examples = True # prints adversarial samples as they are generated, showing the difference between original
using_first_n_samples = False # used when you want to evaluate on a subset of the full eval set.
first_n_samples = 5


In [13]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


### Run to evaluate attacker


In [20]:

if using_mounted_drive:
    data_path =  pathlib.Path(f"{path_to_mounted_folder}/{task}")
    model_path = pathlib.Path(f"{path_to_mounted_folder}/{task}/{victim_model}-512.pth")
    out_dir = pathlib.Path(f"{path_to_mounted_folder}/outputs")

else:
  data_path =  pathlib.Path(f"/content/BODEGA/incrediblAE_public_release/{task}")
  model_path = pathlib.Path(f"/content/BODEGA/incrediblAE_public_release/{task}/{victim_model}-512.pth")
  # out_dir = pathlib.Path("/content/BODEGA/outputs")
  out_dir = pathlib.Path(f"/content/drive/MyDrive/IncredibAE_output")



RESULTS_FILE_NAME = 'results_' + task + '_' + str(targeted) + '_' + attack + '_' + victim_model + '.txt' #stores BODEGA metrics
SUBMISSION_FILE_NAME = 'submission_' + task + '_' + str(targeted) + '_' + attack + '_' + victim_model + '.tsv' #stores original and modified text, to be submitted to shared task organizers

results_path = out_dir / RESULTS_FILE_NAME if out_dir else None
submission_path = out_dir / SUBMISSION_FILE_NAME if out_dir else None

if out_dir:
    if (out_dir / RESULTS_FILE_NAME).exists():
      print(f"Existing results file found. This script will overwrite previous file: {str(results_path)}")
    if submission_path.exists():
      print(f"Existing submission file found. This script will overwrite previous file: {str(submission_path)}")




# Prepare task data
with_pairs = (task == 'FC' or task == 'C19')

# Choose device
print("Setting up the device...")

using_TF = (attack in ['TextFooler', 'BAE'])
if using_TF:
    # Disable GPU usage by TF to avoid memory conflicts
    import tensorflow as tf

    tf.config.set_visible_devices(devices=[], device_type='GPU')

if torch.cuda.is_available():
    print('using GPU')
    victim_device = torch.device("cuda")
    attacker_device = torch.device("cuda")
else:
    victim_device = torch.device("cpu")
    attacker_device = torch.device('cpu')

# Prepare victim
print("Loading up victim model...")
if victim_model == 'BERT':
    victim = VictimCache(model_path, VictimBERT(model_path, task, victim_device))
    readfromfile_generator = BERT_readfromfile_generator
elif victim_model == 'BiLSTM':
    victim = VictimCache(model_path, VictimBiLSTM(model_path, task, victim_device))
    readfromfile_generator = BERT_readfromfile_generator
elif victim_model == 'surprise':
    victim = VictimCache(model_path, VictimRoBERTa(model_path, task, victim_device))
    readfromfile_generator = roberta_readfromfile_generator

# Load data
print("Loading data...")
test_dataset = Dataset.from_generator(readfromfile_generator,
                                      gen_kwargs={'subset': 'attack', 'dir': data_path, 'trim_text': True,
                                                  'with_pairs': with_pairs})
if not with_pairs:
    dataset = test_dataset.map(dataset_mapping)
    dataset = dataset.remove_columns(["text"])
else:
    dataset = test_dataset.map(dataset_mapping_pairs)
    dataset = dataset.remove_columns(["text1", "text2"])

dataset = dataset.remove_columns(["fake"])

# Filter data
if using_first_n_samples:
  dataset = dataset.select(range(first_n_samples))

if targeted:
    dataset = [inst for inst in dataset if inst["y"] == 1 and victim.get_pred([inst["x"]])[0] == inst["y"]]

print("Subset size: " + str(len(dataset)))

# Prepare attack
print("Setting up the attacker...")

# Necessary to bypass the outdated SSL certifiacte on the OpenAttack servers
with no_ssl_verify():
  if using_custom_attacker:
    attacker = MyAttacker()
  else:
    filter_words = OpenAttack.attack_assist.filter_words.get_default_filter_words('english') + [SEPARATOR_CHAR]
    if attack == 'PWWS':
        attacker = OpenAttack.attackers.PWWSAttacker(token_unk=UNK_TEXT, lang='english', filter_words=filter_words)
    elif attack == 'SCPN':
        os.environ["TOKENIZERS_PARALLELISM"] = "false"
        attacker = OpenAttack.attackers.SCPNAttacker(device=attacker_device)
    elif attack == 'TextFooler':
        attacker = OpenAttack.attackers.TextFoolerAttacker(token_unk=UNK_TEXT, lang='english',
                                                           filter_words=filter_words)
    elif attack == 'DeepWordBug':
        attacker = OpenAttack.attackers.DeepWordBugAttacker(token_unk=UNK_TEXT)
    elif attack == 'VIPER':
        attacker = OpenAttack.attackers.VIPERAttacker()
    elif attack == 'GAN':
        attacker = OpenAttack.attackers.GANAttacker()
    elif attack == 'Genetic':
        attacker = OpenAttack.attackers.GeneticAttacker(lang='english', filter_words=filter_words)
    elif attack == 'PSO':
        attacker = OpenAttack.attackers.PSOAttacker(lang='english', filter_words=filter_words)
    elif attack == 'BERTattack':
        attacker = OpenAttack.attackers.BERTAttacker(filter_words=filter_words, use_bpe=False, device=attacker_device)
    elif attack == 'BAE':
        attacker = OpenAttack.attackers.BAEAttacker(device=attacker_device, filter_words=filter_words)
    else:
        attacker = None

# Run the attack
print("Evaluating the attack...")
RAW_FILE_NAME = 'raw_' + task + '_' + str(targeted) + '_' + attack + '_' + victim_model + '.tsv'
raw_path = out_dir / RAW_FILE_NAME if out_dir else None

with no_ssl_verify():
    scorer = BODEGAScore(victim_device, task, align_sentences=True, semantic_scorer="BLEURT", raw_path = raw_path)
    attack_eval = BodegaAttackEval(attacker, victim, language='english', metrics=[
        scorer  # , OpenAttack.metric.EditDistance()
    ])
    start = time.time()
    summary = attack_eval.eval_and_save_tsv(dataset, visualize=visualize_adv_examples, progress_bar=False, tsv_file_path = submission_path)
    end = time.time()
attack_time = end - start
attacker = None

# Remove unused stuff
victim.finalise()
del victim
gc.collect()
torch.cuda.empty_cache()
if "TOKENIZERS_PARALLELISM" in os.environ:
    del os.environ["TOKENIZERS_PARALLELISM"]

# Evaluate
start = time.time()
score_success, score_semantic, score_character, score_BODEGA= scorer.compute()
end = time.time()
evaluate_time = end - start

# Print results
print("Subset size: " + str(len(dataset)))
print("Success score: " + str(score_success))
print("Semantic score: " + str(score_semantic))
print("Character score: " + str(score_character))
print("BODEGA score: " + str(score_BODEGA))
print("Queries per example: " + str(summary['Avg. Victim Model Queries']))
print("Total attack time: " + str(attack_time))
print("Time per example: " + str((attack_time) / len(dataset)))
print("Total evaluation time: " + str(evaluate_time))

if out_dir:
  with open(results_path, 'w') as f:
      f.write("Subset size: " + str(len(dataset)) + '\n')
      f.write("Success score: " + str(score_success) + '\n')
      f.write("Semantic score: " + str(score_semantic) + '\n')
      f.write("Character score: " + str(score_character) + '\n')
      f.write("BODEGA score: " + str(score_BODEGA) + '\n')
      f.write("Queries per example: " + str(summary['Avg. Victim Model Queries']) + '\n')
      f.write("Total attack time: " + str(end - start) + '\n')
      f.write("Time per example: " + str((end - start) / len(dataset)) + '\n')
      f.write("Total evaluation time: " + str(evaluate_time) + '\n')

  print('-')
  print('Bodega metrics saved to', results_path)
  print('Submission file saved to', submission_path)

Existing results file found. This script will overwrite previous file: /content/drive/MyDrive/IncredibAE_output/results_RD_False_RoBERTa_BERT.txt
Setting up the device...
using GPU
Loading up victim model...


/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


Victim caching: file found, loading...
Loading data...
Subset size: 415
Setting up the attacker...
Evaluating the attack...


The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'BleurtSPTokenizer'. 
The class this function is called from is 'BertTokenizer'.


Sample: 1 =====================================================================
Label: 1 (99.95%) --> 0 (82.69%)            |                                   
                                            |                                   
Reports :    # CharlieHebdo suspects        |                                   
        gets # charliehebdo suspects        |                                   
                                            |                                   
killed   http :// t . co / rsl4203bcQ       |                                   
inspired http :// t . co /                  |                                   
                                            |                                   
Damn           , this is like a movie RT @  |                                   
rsl4203bcqdamn , this is like a movie rt @  |                                   
                                            |                                   
HuffingtonPost Reports : # Ch

Token indices sequence length is longer than the specified maximum sequence length for this model (541 > 512). Running this sequence through the model will result in indexing errors


Streaming output truncated to the last 5000 lines.
or hollywood movie ... i think   .@ nytimes |                                   
                                            |                                   
. What the hell . @ nytimes Speak for       |                                   
. what the hell . @ nytimes speak for       |                                   
                                            |                                   
yourself , buddy ! @ nytimes As a true      |                                   
yourself , buddy ! @ nytimes as a true      |                                   
                                            |                                   
iSlamist you ' re supposed to do this .     |                                   
islamist you ' re supposed to do this .     |                                   
                                            | Running Time:            1.7688   
This is exactly what Moo - Ham - Ad did     | Query Exceed

In [ ]:
# from google.colab import runtime
# runtime.unassign()

Your output should look like this.
The custom attack has a very low BODEGA score, suggesting that the attack was not very successful (low success rate and low preservation of meaning).

VictimBERT on PR2:
```
Subset size: 416
Success score: 0.1778846153846154
Semantic score: 0.40792732766351186
Character score: 0.3001644500157
BODEGA score: 0.02308437726605881
Queries per example: 2.1778846153846154
Total attack time: 19.421820878982544
Time per example: 0.04668706942063112
Total evaluation time: 10.617336988449097
```

## Submission Files

Whenever you run an attack on a dataset, a submission_task.tsv file will be saved to your outputs directory. At the end of the test phase, you will need to submit your final attack's submission files to the shared task organisers for evaluation (1 for each dataset * num_victim_classifiers).

The submission file contains 4 pieces of information per attacked text:
1. was the attack successful
2. number of queries to victim model used to generate the adversarial sample
3. the original text
4. the adversarial text (or ATTACK_UNSUCCESSFUL if unsuccessful)

## Final tips:

### Using a subset of eval dataset
Testing your attack on the entire eval dataset can take a while. To speed things up, you can test on the first n samples of the dataset, by setting `using_first_n_samples` to `True`.  

### Running pre-implemented attacks

BODEGA supports a number of pre-existing attacks. Trying these might be useful if you want to:
- compare your performance with existing methods (also reported in the [BODEGA preprint](https://arxiv.org/abs/2303.08032))
- get inspiration from observing their substitutions

To use an existing attack requires only two changes to the code above:
1. set `using_custom_attacker` to `False`
2. set `attack` to the name of a supported attack
(`PWWS`, `SCPN`, `TextFooler`, `DeepWordBug`, `GAN`, `Genetic`, `PSO`, `BERTattack` or`BAE`)

Note that using `BAE` or `TextFooler` will require you to install additional dependencies since they rely on tensorflow:

- tensorflow >= 2.0.0
- tensorflow_hub

https://openattack.readthedocs.io/en/latest/quickstart/installation.html
